In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import random

def random_sampler(filename, k):
    sample = []
    with open(filename, 'rb') as f:
        f.seek(0, 2)
        filesize = f.tell()
        
        random_set = sorted(random.sample(range(filesize), k))
       
        for i in range(k):
            f.seek(random_set[i])
            # Skip current line (because we might be in the middle of a line) 
            f.readline()
            # Append the next line to the sample set 
            sample.append(f.readline().rstrip())

    return sample

In [ ]:
TRAIN_SAMPLES = 5*10**6
train_sample = random_sampler('../input/train.csv', TRAIN_SAMPLES)

In [ ]:
train_sample[0].decode().split(',')

In [ ]:
train_sample_ = [row.decode().split(",") for row in train_sample]

In [ ]:
train = pd.DataFrame(train_sample_)

In [ ]:
del train_sample
del train_sample_

In [ ]:
train_df = pd.read_csv('../input/train.csv', nrows=1)
train.columns = train_df.columns

In [ ]:
train = train.apply(pd.to_numeric)

In [ ]:
train.info()

In [ ]:
train['Demanda_uni_equil'] = np.log1p(train['Demanda_uni_equil'])

In [ ]:
x_cols = train.columns
x_cols = x_cols.drop(['Venta_uni_hoy', 'Venta_hoy', 'Dev_uni_proxima', 'Dev_proxima', 'Demanda_uni_equil'])
print(x_cols)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()

In [ ]:
model.fit(train[x_cols], train['Demanda_uni_equil'])

In [ ]:
test = pd.read_csv('../input/test.csv')
test.info()

In [ ]:
test['Demanda_uni_equil'] = np.expm1(model.predict(test[x_cols]))

In [ ]:
test[['id', 'Demanda_uni_equil']].to_csv('predictions_rf_random_sampling.csv', index=False)